# Wingsail Controller Prototype

In [ ]:
# Install Python dependencies
!pip3 install numpy scipy matplotlib


In [ ]:
# Imports
%matplotlib inline
import numpy as np
import scipy
import matplotlib.pyplot as plt


## Function `foo` Implementation

It takes two integers $a$ and $b$ and returns their sum.

In [ ]:
def foo(a: int, b: int) -> int:
    return a + b


In [ ]:
a = 1
b = 4
c = foo(a, b)
print(f"The result of {a} + {b} is {c}")


## Plot some stuff

In [ ]:
# Reference: https://matplotlib.org/stable/gallery/lines_bars_and_markers/fill_between_demo.html#example-confidence-bands
N = 21
x = np.linspace(0, 10, 11)
y = [3.9, 4.4, 10.8, 10.3, 11.2, 13.1, 14.1, 9.9, 13.9, 15.1, 12.5]

# fit a linear curve and estimate its y-values and their error.
a, b = np.polyfit(x, y, deg=1)
y_est = a * x + b
y_err = x.std() * np.sqrt(1 / len(x) + (x - x.mean()) ** 2 / np.sum((x - x.mean()) ** 2))

fig, ax = plt.subplots()
ax.plot(x, y_est, "-")
ax.fill_between(x, y_est - y_err, y_est + y_err, alpha=0.2)
ax.plot(x, y, "o", color="tab:brown")
